In [1]:
from pathlib import Path

import os
import requests
import time 
import sys

# For saving it as a csv file 
import pandas as pd 

# Add the src directory to the system path
src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path: sys.path.append(src_path)
# Import your fetch function
from utils import *

# Lösung für Notebooks: 
# In Jupyter: Arbeitsverzeichnis (z. B. Projektordner)
project_root = Path().resolve().parents[0]
sys.path.append(str(project_root))
# Path wo die roh daten gespeichert sind 
path_data_raw       = project_root / "data" / "raw"
# Path wo die processed data gespeichert wird 
path_data_processed = project_root / "data" / "processed"
# Path wo die processed data gespeichert wird 
path_plots           = project_root / "reports" / "plots"


print(project_root)
print(path_data_processed)
print(path_data_raw)

# Pfade sicherstellen für die process data files und fuer die plot files im reports Folder 
Path("data/processed").mkdir(parents=True, exist_ok=True)
Path("reports/plots").mkdir(parents=True, exist_ok=True)



/Users/tillo/Repositorys/stock-price-rnn
/Users/tillo/Repositorys/stock-price-rnn/data/processed
/Users/tillo/Repositorys/stock-price-rnn/data/raw


In [2]:

"""Lösung nur für Python Scripte weil hier __file__ erkannt wird 
project_root = Path(__file__).resolve().parents[1]

data_folder = Path(__file__).resolve().parents[1] / "data" / "raw"

    # Create folder if it doesn't exist
data_folder.mkdir(parents=True, exist_ok=True)
"""




# Finde alle CSV-Dateien
csv_files = list_csv_files(path_data_raw)

#print(data_folder)
#print(csv_files)

# Lade sie in ein Dictionary mit DataFrames
dfs = load_csvs_as_dfs(csv_files, path_data_raw)

# Zugriff auf einen bestimmten DataFrame:
print(dfs.keys())  # z. B. df_apple_stock, df_dax_2020 etc.

# Erstellen einer liste der keys der aktien 
keys_list = list(dfs.keys())
print(keys_list[0])



dict_keys(['df_googl_last_1000_days', 'df_mbg.dex_last_1000_days', 'df_aapl_last_1000_days', 'df_msft_last_1000_days'])
df_googl_last_1000_days


In [3]:
processed_dfs = {}

for name, df in dfs.items():
    df = add_moving_average(df)
    df = add_ema(df)
    df = add_lowess(df)
    df = add_savgol_filter(df)
    df = add_local_peaks(df, distance=50)
    df = add_local_valleys(df, distance=50)
    df = decompose_seasonality(df)
    df = analyze_volume_peaks(df)


    # Neue einzel Pfade definieren
    plot = path_plots / f"{name}.png"
    data = path_data_processed / f"{name}.csv"

    # Plot und speichern
    plot_price_with_indicators(df, symbol=name, save_path=plot, show=False)
    plot_trend_seasonal_resid(df, symbol=name, save_path=plot, show=False)

    # Daten speichern
    df.to_csv(data, index=True)
    
    processed_dfs[name] = df


print("Verarbeitung abgeschlossen für:", list(processed_dfs.keys()))


Verarbeitung abgeschlossen für: ['df_googl_last_1000_days', 'df_mbg.dex_last_1000_days', 'df_aapl_last_1000_days', 'df_msft_last_1000_days']


In [4]:
# Test df für prediction with time series model 
df_test = processed_dfs['df_aapl_last_1000_days']

df_test.head()

,open,high,low,close,volume,sma_20,ema_20,lowess,savgol,is_peak,is_valley,trend,seasonal,resid,volume_sma_20,high_volume
Date,,,,,,,,,,,,,,,,
2021-07-06,140.070,143.15,140.0700,142.02,108181793,NaN,142.020000,144.564033,143.162349,False,False,156.978788,-14.550177,-0.408611,NaN,False
2021-07-07,143.535,144.89,142.6600,144.57,104911589,NaN,142.262857,144.650988,143.723177,False,False,156.985385,-12.329170,-0.086215,NaN,False
2021-07-08,141.580,144.06,140.6650,143.24,105575458,NaN,142.355918,144.737971,144.238438,False,False,156.991986,-13.401721,-0.350264,NaN,False
2021-07-09,142.750,145.65,142.6522,145.11,99890800,NaN,142.618212,144.825031,144.708133,False,False,156.998591,-11.593050,-0.295540,NaN,False
2021-07-12,146.210,146.32,144.0000,144.50,76299719,NaN,142.797430,145.087333,145.132262,False,False,157.005201,-12.327016,-0.178185,NaN,False


In [5]:
# Umgang mit unterschiedlichen Zeitreihen und anwendung von
# ARMA; ARIMA oder SARIMA Modell
################ ARMA #################
#### ist eigentlich eine ARIMA modell mit d=o
from statsmodels.tsa.arima.model import ARIMA
# ARMA-Modell (AR=2, MA=2)
model = ARIMA(df_test, order=(2, 0, 2)) # d=0 → stationäre Zeitreihe
result = model.fit()
# Prognose ausgeben
print(result.summary())



ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:

################ ARIMA #################
# ARIMA-Modell (AR=2, I=1, MA=2) → 1-fache Differenzierung für Stationarität
model = ARIMA(data, order=(2, 1, 2))
result = model.fit()
# Vorhersagen für die nächsten 12 Zeitpunkte
forecast = result.forecast(steps=12)
print(forecast)
################ SARIMA #################
from statsmodels.tsa.statespace.sarimax import SARIMAX
# SARIMA-Modell (z. B. mit jährlicher Saisonalität für Monatsdaten)
model = SARIMAX(data, order=(2, 1, 2), seasonal_order=(1, 1, 1, 12))
result = model.fit()
# Vorhersage für die nächsten 12 Monate
forecast = result.get_forecast(steps=12)
print(forecast.predicted_mean)

die nächsten möglichen schritte: 

1. Grundlegende statistische Kennzahlen berechnen
Mittelwerte (Mean) z.B. für close-Preise über einen Zeitraum (Tag, Woche, Monat)

Median, Standardabweichung, Varianz — um die Volatilität einzuschätzen

2. Gleitende Durchschnitte (Moving Averages)
Berechnung von gleitenden Durchschnitten wie dem Simple Moving Average (SMA) oder Exponential Moving Average (EMA)

Hilft, Trends zu glätten und rauschen zu reduzieren

3. Glättung der Kurve
Mit Moving Averages oder Lowess-Glättung (LOcally WEighted Scatterplot Smoothing)

Alternativ auch mit Savitzky-Golay-Filter

4. Lokale Maxima und Minima finden
Suche nach Hochs und Tiefs in der Zeitreihe, z.B. mit scipy.signal.find_peaks

Definiere Abstand oder Minimum-Distanz (z.B. 1 Monat = ca. 20 Börsentage) zwischen Peaks

Erkenne Unterstützungs- und Widerstandslinien

5. Saisonale Muster und Zeitreihenanalyse
Analyse mit Rolling Windows oder Seasonal Decomposition (z.B. STL-Decomposition)

Untersuchung von Trends, Saisonalität, Residuen

Beispiel: Hat der Kurs z.B. immer im Januar einen Anstieg?

6. Volumen-Analyse
Analyse des Handelsvolumens in Kombination mit Kursbewegungen (z.B. Volumen bei Hochs und Tiefs)

7. Visualisierung
Plotten der Rohdaten, Moving Averages, Peaks/Valleys, Saisonalität

Plot mit matplotlib, seaborn oder plotly


